In [141]:
# load relevant packages, set some display parameters
%reset -f

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time

from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, RepeatedKFold
from math import sqrt
from scipy import stats
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, learning_curve
#import ifsg
import glob, os #I ADDED
from sklearn.svm import SVR, LinearSVR #I ADDED
from sklearn.tree import DecisionTreeRegressor #I ADDED
import pickle
import re
import random
import os
import warnings
import seaborn as sns
import gc
from sklearn.decomposition import PCA

warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

#ifsg.__version__

In [142]:
# # set paths
# x=os.path.splitext(os.path.split(os.path.splitext(os.path.split(os.getcwd())[0])[0])[0])
# path_dataset = str(x[0])+os.sep+"Data"+os.sep+"Datasets"+os.sep+"observations"

# print('path_dataset = ', path_dataset)

In [143]:
# load data
path_dataset = 'W:\\Work\\Assense\\ML\\chars_file'
observation_filename = os.path.join(path_dataset,'data_from_firestore_2403.csv')
observation_table = pd.read_csv(observation_filename)#,usecols=range(9))#, index_col=0,skiprows=[0,1,2])#,usecols=range(12))
observation_table.head()
#observation_table.replace('Precision','Thoroughness',inplace=True)

,id,company_name,Time Management,Teamwork,Quantitative reasoning,Quantity vs Quality,Adaptability to changes,Agreeableness,Decisiveness,Purposefulness Vs. Hesitant,Adaptability to Changes,Risk Taking,Stress Tolerance,Attention: Coping with Distractions,Goal orientation,Coping with stress,Methodicalness,Performance IQ,Thoroughness,Empathy,Processing Speed,Performance under stress,Processing speed,Perceptual Organization,Precision,Monitoring,Orderliness,Memory,Performance under Stress,Self Monitoring,Working memory,Self-efficacy,Spatial reasoning,Verbal reasoning,Appropriate Behaviour Understanding,Quantitative Reasoning,Conscientiousness,Cognitive ability total score,Planning,Verbal Reasoning,Performance IQ,Emotional Understanding,Collegial Behavior,Intelligence Quotient
0,5beadc3169056d543c06c4e66,Actiview,5.0,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,5.0,NaN,NaN,5.0
1,12345678,Actiview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5C45C837378D814141D7B22C,Actiview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,6.0,NaN,6.0,6.0,NaN,NaN,NaN
3,5b421df4ff36d1e2d5a70e77,Actiview,1.0,3.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,6.0,3.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,1.0,NaN,NaN,1.0
4,5bb3c16069ad6d50f0356ab7,Actiview,3.0,4.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,3.0,NaN,NaN,3.0


In [144]:
observation_table.replace({'Precision':'Thoroughness',
                     'Decisiveness Vs. Hesitant':'Decisiveness',
                     'Purposefulness vs. Hesitant':'Decisiveness',
                     'Monitor and review _ process and product Monitoring':'Self Monitoring',
                     'Monitor And Review | Process And Product Monitoring':'Self Monitoring',
                     'Monitor and review – process and product Monitoring':'Self Monitoring',
                     'Quality Vs. Quantity':'Quantity vs Quality',
                     'Time management':'Time Management',
                     '(stress tolerance) Reaction to stress and difficulty':'Stress Tolerance',
                     'Stress Tolerance | Reaction To Stress And Difficulty':'Stress Tolerance'},inplace=True)

In [193]:
Mit_db = observation_table.loc[observation_table['company_name'] == 'Mitsubishi']

In [198]:
Mit_db

,id,company_name,Time Management,Teamwork,Quantitative reasoning,Quantity vs Quality,Adaptability to changes,Agreeableness,Decisiveness,Purposefulness Vs. Hesitant,Adaptability to Changes,Risk Taking,Stress Tolerance,Attention: Coping with Distractions,Goal orientation,Coping with stress,Methodicalness,Performance IQ,Thoroughness,Empathy,Processing Speed,Performance under stress,Processing speed,Perceptual Organization,Precision,Monitoring,Orderliness,Memory,Performance under Stress,Self Monitoring,Working memory,Self-efficacy,Spatial reasoning,Verbal reasoning,Appropriate Behaviour Understanding,Quantitative Reasoning,Conscientiousness,Cognitive ability total score,Planning,Verbal Reasoning,Performance IQ,Emotional Understanding,Collegial Behavior,Intelligence Quotient
726,5c1c54d7da0e492bb4580c7b,Mitsubishi,4.000000,NaN,-1.0,NaN,NaN,NaN,6.000000,NaN,4.000000,2.000000,NaN,4.000000,5.333333,NaN,4.666667,NaN,4.666667,-1.00,5.9,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,3.000000,3.333333,NaN,NaN,5.3,NaN,-1.0000,NaN,-1.0,5.5,4.666667,5.8,6.0,-1.0,-1.00,NaN
727,5c1c55cfda0e492bb4580c7d,Mitsubishi,4.500000,NaN,-1.0,NaN,NaN,NaN,4.000000,NaN,5.000000,1.000000,NaN,6.000000,4.000000,NaN,5.000000,NaN,5.500000,-1.00,4.2,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,1.500000,4.500000,NaN,NaN,4.1,NaN,-1.0000,NaN,-1.0,5.6,5.000000,5.3,4.6,-1.0,-1.00,NaN
728,5c2315fb58497715f43f3dd7,Mitsubishi,3.500000,NaN,-1.0,NaN,NaN,NaN,2.500000,NaN,2.000000,4.500000,NaN,3.000000,1.500000,NaN,3.000000,NaN,2.500000,-1.00,3.4,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,1.000000,2.000000,NaN,NaN,3.1,NaN,-1.0000,NaN,-1.0,3.9,2.000000,2.5,3.5,-1.0,-1.00,NaN
729,5c2316ec58497715f43f3dd8,Mitsubishi,3.500000,NaN,-1.0,NaN,NaN,NaN,4.000000,NaN,2.500000,5.000000,NaN,3.500000,1.500000,NaN,2.000000,NaN,2.000000,-1.00,4.1,NaN,NaN,NaN,NaN,NaN,2.500000,NaN,1.000000,2.500000,NaN,NaN,3.6,NaN,-1.0000,NaN,-1.0,5.5,1.000000,5.1,3.7,-1.0,-1.00,NaN
730,5c380026ab15e621f457e791,Mitsubishi,4.666667,NaN,-1.0,NaN,NaN,NaN,2.000000,NaN,1.333333,3.666667,NaN,2.000000,1.666667,NaN,2.000000,NaN,4.000000,1.25,4.6,NaN,NaN,NaN,NaN,NaN,1.666667,NaN,1.333333,3.000000,NaN,NaN,4.2,NaN,3.1875,NaN,2.5,5.0,3.666667,4.3,4.8,4.5,5.25,NaN
731,5c4a955bcd053b079445153a,Mitsubishi,4.600000,NaN,-1.0,NaN,NaN,NaN,4.000000,NaN,5.250000,1.000000,NaN,5.666667,4.000000,NaN,5.250000,NaN,5.400000,-1.00,4.8,NaN,NaN,NaN,NaN,NaN,4.200000,NaN,4.600000,4.200000,4.9,NaN,5.2,NaN,-1.0000,NaN,-1.0,5.1,4.000000,4.8,5.5,-1.0,-1.00,NaN
732,5c4aa60e9dd0fb08d0c4545c,Mitsubishi,3.000000,NaN,-1.0,NaN,NaN,NaN,3.000000,NaN,4.000000,5.500000,NaN,4.000000,2.750000,NaN,2.750000,NaN,2.750000,-1.00,3.0,NaN,NaN,NaN,NaN,NaN,1.600000,NaN,1.400000,3.000000,4.1,NaN,3.8,NaN,-1.0000,NaN,-1.0,4.0,3.800000,3.9,3.9,-1.0,-1.00,NaN
733,5c4aa6229dd0fb08d0c4545d,Mitsubishi,4.000000,NaN,-1.0,NaN,NaN,NaN,6.000000,NaN,1.333333,5.000000,NaN,3.500000,3.000000,NaN,3.000000,NaN,4.333333,-1.00,3.2,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,1.666666,2.333333,3.8,NaN,4.6,NaN,-1.0000,NaN,-1.0,4.1,3.000000,3.2,4.4,-1.0,-1.00,NaN
734,5c4aa6349dd0fb08d0c4545e,Mitsubishi,2.000000,NaN,-1.0,NaN,NaN,NaN,3.800000,NaN,2.000000,4.500000,NaN,4.500000,1.750000,NaN,1.500000,NaN,2.900000,-1.00,3.8,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,3.000000,2.300000,4.0,NaN,3.5,NaN,-1.0000,NaN,-1.0,3.8,2.200000,3.8,3.9,-1.0,-1.00,NaN
735,5c4e4503f01357149cc444ed,Mitsubishi,2.000000,NaN,-1.0,NaN,NaN,NaN,1.800000,NaN,1.333333,4.500000,NaN,2.500000,1.000000,NaN,1.000000,NaN,2.600000,-1.00,3.9,NaN,NaN,NaN,NaN,NaN,3.333333,NaN,2.000000,2.300000,3.2,NaN,1.8,NaN,-1.0000,NaN,-1.0,2.7,1.800000,3.3,3.5,-1.0,-1.00,NaN


In [202]:
Mit_db_r= Mit_db[['id','Cognitive ability total score', 'Spatial reasoning', 'Verbal reasoning', 'Processing speed', 
                 'Performance IQ', 'Working memory', 'Quantitative Reasoning', 'Planning', 'Time Management', 'Monitoring',
                 'Goal orientation', 'Thoroughness','Orderliness', 'Methodicalness', 'Performance under Stress','Risk Taking', 
                 'Adaptability to Changes','Decisiveness']]

In [207]:
Mit_db_r
#.loc[Mit_db_r['id'] == '5C35A1D0B5997B32B4CEAA9B']

,id,Cognitive ability total score,Spatial reasoning,Verbal reasoning,Processing speed,Performance IQ,Working memory,Quantitative Reasoning,Planning,Time Management,Monitoring,Goal orientation,Thoroughness,Orderliness,Methodicalness,Performance under Stress,Risk Taking,Adaptability to Changes,Decisiveness
726,5c1c54d7da0e492bb4580c7b,5.5,5.3,NaN,NaN,6.0,NaN,NaN,4.666667,4.000000,NaN,5.333333,4.666667,3.000000,4.666667,3.000000,2.000000,4.000000,6.000000
727,5c1c55cfda0e492bb4580c7d,5.6,4.1,NaN,NaN,4.6,NaN,NaN,5.000000,4.500000,NaN,4.000000,5.500000,3.000000,5.000000,1.500000,1.000000,5.000000,4.000000
728,5c2315fb58497715f43f3dd7,3.9,3.1,NaN,NaN,3.5,NaN,NaN,2.000000,3.500000,NaN,1.500000,2.500000,3.000000,3.000000,1.000000,4.500000,2.000000,2.500000
729,5c2316ec58497715f43f3dd8,5.5,3.6,NaN,NaN,3.7,NaN,NaN,1.000000,3.500000,NaN,1.500000,2.000000,2.500000,2.000000,1.000000,5.000000,2.500000,4.000000
730,5c380026ab15e621f457e791,5.0,4.2,NaN,NaN,4.8,NaN,NaN,3.666667,4.666667,NaN,1.666667,4.000000,1.666667,2.000000,1.333333,3.666667,1.333333,2.000000
731,5c4a955bcd053b079445153a,5.1,5.2,NaN,NaN,5.5,4.9,NaN,4.000000,4.600000,NaN,4.000000,5.400000,4.200000,5.250000,4.600000,1.000000,5.250000,4.000000
732,5c4aa60e9dd0fb08d0c4545c,4.0,3.8,NaN,NaN,3.9,4.1,NaN,3.800000,3.000000,NaN,2.750000,2.750000,1.600000,2.750000,1.400000,5.500000,4.000000,3.000000
733,5c4aa6229dd0fb08d0c4545d,4.1,4.6,NaN,NaN,4.4,3.8,NaN,3.000000,4.000000,NaN,3.000000,4.333333,2.000000,3.000000,1.666666,5.000000,1.333333,6.000000
734,5c4aa6349dd0fb08d0c4545e,3.8,3.5,NaN,NaN,3.9,4.0,NaN,2.200000,2.000000,NaN,1.750000,2.900000,1.000000,1.500000,3.000000,4.500000,2.000000,3.800000
735,5c4e4503f01357149cc444ed,2.7,1.8,NaN,NaN,3.5,3.2,NaN,1.800000,2.000000,NaN,1.000000,2.600000,3.333333,1.000000,2.000000,4.500000,1.333333,1.800000


In [209]:
Mit_db_r.to_excel('Mit_db_r.xlsx')

In [140]:
observation_table.keys()

Index(['id', 'company_name', 'Time Management', 'Teamwork', 'Quantitative reasoning', 'Quantity vs Quality', 'Adaptability to changes', 'Agreeableness', 'Decisiveness', 'Purposefulness Vs. Hesitant', 'Adaptability to Changes', 'Risk Taking', 'Stress Tolerance', 'Attention: Coping with Distractions', 'Goal orientation', 'Coping with stress', 'Methodicalness', 'Performance IQ ', 'Thoroughness', 'Empathy', 'Processing Speed', 'Performance under stress', 'Processing speed', 'Perceptual Organization', 'Precision', 'Monitoring', 'Orderliness', 'Memory', 'Performance under Stress', 'Self Monitoring', 'Working memory', 'Self-efficacy', 'Spatial reasoning', 'Verbal reasoning', 'Appropriate Behaviour Understanding', 'Quantitative Reasoning', 'Conscientiousness', 'Cognitive ability total score', 'Planning', 'Verbal Reasoning', 'Performance IQ', 'Emotional Understanding', 'Collegial Behavior', 'Intelligence Quotient'], dtype='object')

In [59]:
half = observation_table.shape[0]
observation_table.isnull().sum()/half*100


id                                      0.000000
company_name                            0.000000
Time Management                         3.934010
Teamwork                               40.862944
Quantitative reasoning                 77.664975
Quantity vs Quality                    79.441624
Adaptability to changes                81.725888
Agreeableness                          97.715736
Decisiveness                           65.609137
Purposefulness Vs. Hesitant            37.817259
Adaptability to Changes                94.796954
Risk Taking                            77.664975
Stress Tolerance                       55.837563
Attention: Coping with Distractions    65.736041
Goal orientation                       65.609137
Coping with stress                     97.715736
Methodicalness                         65.609137
Performance IQ                         98.604061
Thoroughness                           65.609137
Empathy                                70.050761
Processing Speed    

In [53]:
#Export relevant id
#observation_table['id'].to_csv("id.csv", index = False)


In [76]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]


In [108]:
import json
file = "W:\\Work\\Assense\\ML\\DataFrames\\ang_25_03_19.json"
with open(file) as train_file:
    dict_train = json.load(train_file)
sol_ac=[]
df = pd.DataFrame.from_dict(dict_train)
print(df.shape)

for col in list(df.columns):
    if re.search('ang.s_sAccuracy\..', col):
        sol_ac.append(col)

sol_ac.sort(key=natural_keys)
df_gnr= df[sol_ac]
df_ang = df_gnr.sum()
df_ang/15

(448, 438)


ang.s_sAccuracy.0     22.000000
ang.s_sAccuracy.1     22.600000
ang.s_sAccuracy.2     18.000000
ang.s_sAccuracy.3     26.500000
ang.s_sAccuracy.4     14.700000
ang.s_sAccuracy.5     21.800000
ang.s_sAccuracy.6     21.466667
ang.s_sAccuracy.7     21.800000
ang.s_sAccuracy.8     24.300000
ang.s_sAccuracy.9     14.700000
ang.s_sAccuracy.10    18.566667
ang.s_sAccuracy.11    10.100000
ang.s_sAccuracy.12    17.733333
ang.s_sAccuracy.13    19.766667
ang.s_sAccuracy.14     9.800000
ang.s_sAccuracy.15    20.400000
ang.s_sAccuracy.16    11.666667
dtype: float64

In [107]:
file1 = "W:\\Work\\Assense\\ML\\DataFrames\\pzl_24_03_19.json"
with open(file1) as train_file:
    dict_train = json.load(train_file)
sol_ac=[]
df = pd.DataFrame.from_dict(dict_train)
print(df.shape)

for col in list(df.columns):
    if re.search('pzl.r_sAccuracy\..', col):
        sol_ac.append(col)

sol_ac.sort(key=natural_keys)
df_gnr= df[sol_ac]
df_pzl = df_gnr.sum()
df_pzl/24

(453, 1637)


pzl.r_sAccuracy.0      8.625000
pzl.r_sAccuracy.1     16.708333
pzl.r_sAccuracy.2     15.291667
pzl.r_sAccuracy.3     16.583333
pzl.r_sAccuracy.4     18.458333
pzl.r_sAccuracy.5     15.750000
pzl.r_sAccuracy.6     17.250000
pzl.r_sAccuracy.7     13.916667
pzl.r_sAccuracy.8     14.958333
pzl.r_sAccuracy.9     17.666667
pzl.r_sAccuracy.10    13.541667
pzl.r_sAccuracy.11    17.458333
pzl.r_sAccuracy.12    15.666667
pzl.r_sAccuracy.13    16.333333
pzl.r_sAccuracy.14    11.250000
pzl.r_sAccuracy.15    12.750000
pzl.r_sAccuracy.16    18.583333
pzl.r_sAccuracy.17    17.625000
pzl.r_sAccuracy.18    12.541667
pzl.r_sAccuracy.19    15.666667
pzl.r_sAccuracy.20    17.583333
pzl.r_sAccuracy.21    18.375000
pzl.r_sAccuracy.22    16.083333
pzl.r_sAccuracy.23    14.541667
dtype: float64

In [114]:
file2 = "W:\\Work\\Assense\\ML\\DataFrames\\shp_24_03_19.json"
with open(file2) as train_file:
    dict_train = json.load(train_file)
sol_ac=[]
df = pd.DataFrame.from_dict(dict_train)
print(df.shape)

for col in list(df.columns):
    if re.search('shp.r_sAccuracy_sum_scaled', col):
        sol_ac.append(col)

sol_ac.sort(key=natural_keys)
df_gnr= df[sol_ac]
df_gnr.head()


(432, 473)


,shp.r_sAccuracy_sum_scaled
5BB3C16069AD6D50F0356AB7,-2.490778
5BB4FA8BB9CD476B345D474D,0.260176
5BB5D8FD5573C7458C89B43B,-0.918804
5BB5DF015573C7458C89B440,-1.704791
5BBA1F9120AB8A117956A926,-0.918804


In [125]:
file3 = "W:\\Work\\Assense\\ML\\DataFrames\\pcs_24_03_19.json"
with open(file3) as train_file:
    dict_train = json.load(train_file)
sol_ac=[]
df = pd.DataFrame.from_dict(dict_train)
print(df.shape)

#if re.search('pcs.r_fAccuracy\..', col):

for col in list(df.columns):
    if re.search('racy*', col):
        sol_ac.append(col)

sol_ac.sort(key=natural_keys)
df_gnr= df[sol_ac]
df_pcs = df_gnr.sum()
df_pcs/87

(166, 3640)


pcs.r_fAccuracy.0                  1.908046e+00
pcs.r_fAccuracy.1                  1.908046e+00
pcs.r_fAccuracy.2                  1.823755e+00
pcs.r_fAccuracy.3                  1.858238e+00
pcs.r_fAccuracy.4                  1.888889e+00
pcs.r_fAccuracy.5                  1.885057e+00
pcs.r_fAccuracy.6                  1.885057e+00
pcs.r_fAccuracy.7                  1.888889e+00
pcs.r_fAccuracy.8                  1.896552e+00
pcs.r_fAccuracy.9                  1.865900e+00
pcs.r_fAccuracy.10                 1.896552e+00
pcs.r_fAccuracy.11                 1.896552e+00
pcs.r_fAccuracy.12                 1.865900e+00
pcs.r_fAccuracy.13                 1.877395e+00
pcs.r_fAccuracy.14                 1.640805e+00
pcs.r_fAccuracy.15                 1.830460e+00
pcs.r_fAccuracy.16                 1.744253e+00
pcs.r_fAccuracy.17                 1.813218e+00
pcs.r_fAccuracy.18                 1.747126e+00
pcs.r_fAccuracy.19                 1.660920e+00
pcs.r_fAccuracy.20                 1.473

In [91]:
# pivot table
#scores = pd.pivot_table(observation_table,values='TotalScore',index=['Question','MissionId'],columns='Username',aggfunc=np.max)
#scores.head()